##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agente REINFORCE

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/6_reinforce_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/6_reinforce_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/6_reinforce_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/agents/tutorials/6_reinforce_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Introdução

Este exemplo mostra como treinar um agente [REINFORCE](https://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf) no ambiente Cartpole usando a biblioteca TF-Agents, de maneira semelhante ao [tutorial do DQN](1_dqn_tutorial.ipynb).

![Ambiente Cartpole](images/cartpole.png)

Ele guiará você por todos os componentes de um pipeline de Aprendizado por Reforço (RL) para treinamento, avaliação e coleta de dados.


## Configuração

Se você ainda não instalou as seguintes dependências, execute:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet xvfbwrapper


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.reinforce import reinforce_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import py_tf_eager_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

## Hiperparâmetros

In [ ]:
env_name = "CartPole-v0" # @param {type:"string"}
num_iterations = 250 # @param {type:"integer"}
collect_episodes_per_iteration = 2 # @param {type:"integer"}
replay_buffer_capacity = 2000 # @param {type:"integer"}

fc_layer_params = (100,)

learning_rate = 1e-3 # @param {type:"number"}
log_interval = 25 # @param {type:"integer"}
num_eval_episodes = 10 # @param {type:"integer"}
eval_interval = 50 # @param {type:"integer"}

## Ambiente

Os ambientes no RL representam a tarefa ou o problema que estamos tentando resolver. Os ambientes padrão podem ser facilmente criados no TF-Agents usando `suites`. Temos `suites` diferentes para carregar ambientes de origens como OpenAI Gym, Atari, DM Control, etc., considerando um nome de ambiente de string.

Agora, vamos carregar o ambiente CartPole a partir da suíte do OpenAI Gym.

In [ ]:
env = suite_gym.load(env_name)

Podemos renderizar esse ambiente para ver como ele fica. Um pêndulo que balança livremente é acoplado a um carrinho. O objetivo é mover o carrinho para a direita ou para a esquerda a fim de manter o pêndulo em pé.

In [ ]:
#@test {"skip": true}
env.reset()
PIL.Image.fromarray(env.render())

O statement `time_step = environment.step(action)` aceita a `action` no ambiente. A tupla `TimeStep` retornada contém a próxima observação e recompensa do ambiente para a ação. Os métodos `time_step_spec()` e `action_spec()` no ambiente retornam as especificações (tipos, formatos e limites) do `time_step` e `action`, respectivamente.

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

Portanto, vemos que essa observação é um array de 4 floats: a posição e a velocidade do carrinho, e a posição angular e a velocidade do pêndulo. Como somente duas ações são possíveis (mover para a esquerda ou para a direita), `action_spec` é um escalar em que 0 significa "mover para a esquerda" e 1 significa "mover para a direita".

In [ ]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Geralmente, criamos dois ambientes: um para treinamento e um para avaliação. A maioria dos ambientes são escritos em python puro, mas podem ser facilmente convertidos para TensorFlow usando o wrapper `TFPyEnvironment`. A API do ambiente original usa arrays do numpy, e o `TFPyEnvironment` os converte para/de `Tensors`, para você interagir mais facilmente com as políticas e os agentes do TensorFlow.


In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agente

O algoritmo que usamos para resolver um problema de RL é representado como um `Agent`. Além do agente REINFORCE, o TF-Agents oferece implementações padrão de uma variedade de `Agents`, como [DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf), [DDPG](https://arxiv.org/pdf/1509.02971.pdf), [TD3](https://arxiv.org/pdf/1802.09477.pdf), [PPO](https://arxiv.org/abs/1707.06347) e [SAC](https://arxiv.org/abs/1801.01290).

Para criar um agente REINFORCE, primeiro precisamos de uma `Actor Network` que possa aprender a prever a ação a partir de uma observação do ambiente.

Podemos criar facilmente uma `Actor Network` usando as especificações das observações e ações. Podemos especificar as camadas na rede em que, nesse exemplo, é o argumento `fc_layer_params` definido como uma tupla de `ints` que representam os tamanhos de cada camada oculta (veja a seção Hiperparâmetros acima).


In [ ]:
actor_net = actor_distribution_network.ActorDistributionNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

Também precisamos de um `optimizer` para treinar a rede que acabamos de criar e uma variável `train_step_counter` para acompanhar quantas vezes a rede foi atualizada.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

tf_agent = reinforce_agent.ReinforceAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    actor_network=actor_net,
    optimizer=optimizer,
    normalize_returns=True,
    train_step_counter=train_step_counter)
tf_agent.initialize()

## Políticas

No TF-Agents, as políticas representam a noção padrão das políticas no RL: a partir de um `time_step`, é produzida uma ação ou distribuição sobre ações. O principal método é `policy_step = policy.action(time_step)`, em que `policy_step` é uma tupla nomeada `PolicyStep(action, state, info)`. A `policy_step.action` é a `action` que será aplicada ao ambiente, o `state` representa o estado das políticas stateful (RNN), e a `info` pode conter informações auxiliares, como probabilidades log de ações.

Os agentes contêm duas políticas: a principal que é usada para avaliação/implantação (agent.policy) e outra que é usada para a coleta de dados (agent.collect_policy).

In [ ]:
eval_policy = tf_agent.policy
collect_policy = tf_agent.collect_policy

## Métricas e avaliação

A métrica mais comum usada para avaliar uma política é o retorno médio. O retorno é a soma das recompensas obtidas ao executar uma política em um ambiente para um episódio, e geralmente calculamos isso em alguns episódios. Podemos computar a métrica de retorno médio da seguinte maneira:


In [ ]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# Please also see the metrics module for standard implementations of different
# metrics.

## Buffer de replay

Para rastrear os dados coletados do ambiente, vamos usar o [Reverb](https://deepmind.com/research/open-source/Reverb), um sistema de replay eficiente, extensível e fácil de usar do Deepmind. Ele armazena dados de experiências quando coletamos trajetórias e é consumido durante o treinamento.

Esse buffer de replay é construído usando as especificações que descrevem os tensores a serem armazenados, que podem ser obtidas do agente usando `tf_agent.collect_data_spec`.

In [ ]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      tf_agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
      replay_buffer_signature)
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    table_name=table_name,
    sequence_length=None,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddEpisodeObserver(
    replay_buffer.py_client,
    table_name,
    replay_buffer_capacity
)

Para a maioria dos agentes, a `collect_data_spec` é uma tupla nomeada `Trajectory` que contém a observação, ação, recompensa etc.

## Coleta de dados

Conforme o REINFORCE aprende com episódios inteiros, definimos uma função para coletar um episódio usando a política de coleta de dados especificada e salvar os dados (observações, ações, recompensas etc.) como trajetórias no buffer de replay. Aqui, vamos usar o "PyDriver" para executar o loop de coleta de experiência. Saiba mais sobre o driver TF Agents no nosso [tutorial sobre drivers](https://www.tensorflow.org/agents/tutorials/4_drivers_tutorial).

In [ ]:
#@test {"skip": true}

def collect_episode(environment, policy, num_episodes):

  driver = py_driver.PyDriver(
    environment,
    py_tf_eager_policy.PyTFEagerPolicy(
      policy, use_tf_function=True),
    [rb_observer],
    max_episodes=num_episodes)
  initial_time_step = environment.reset()
  driver.run(initial_time_step)

## Treinamento do agente

O loop de treinamento envolve tanto coletar os dados do ambiente quanto otimizar as redes do agente. Ao longo do caminho, vamos avaliar ocasionalmente a política do agente para ver como está o desempenho.

A execução do código a seguir levará cerca de 3 minutos.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
tf_agent.train = common.function(tf_agent.train)

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few episodes using collect_policy and save to the replay buffer.
  collect_episode(
      train_py_env, tf_agent.collect_policy, collect_episodes_per_iteration)

  # Use data from the buffer and update the agent's network.
  iterator = iter(replay_buffer.as_dataset(sample_batch_size=1))
  trajectories, _ = next(iterator)
  train_loss = tf_agent.train(experience=trajectories)  

  replay_buffer.clear()

  step = tf_agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

## Visualização


### Plots

Podemos plotar o retorno em comparação com os passos dos globais para ver o desempenho do nosso agente. Em `Cartpole-v0`, o ambiente dá uma recompensa de +1 a cada timestep em que o pêndulo permanece em pé e, como o número máximo de passos é 200, o retorno máximo possível também é 200.

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim(top=250)

### Vídeos

É útil visualizar o desempenho de um agente ao renderizar o ambiente a cada passo. Antes de fazer isso, vamos criar uma função para incorporar vídeos neste colab.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

O código a seguir visualiza a política do agente por alguns episódios:

In [ ]:
num_episodes = 3
video_filename = 'imageio.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_py_env.render())
    while not time_step.is_last():
      action_step = tf_agent.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_py_env.render())

embed_mp4(video_filename)